# 1. Introduction to the Workforce Allocation Problem (WAP)

Welcome to this series on the **Workforce Allocation Problem (WAP)**. In this first notebook, we will introduce the problem's core concepts, constraints, and objectives. We'll explore a sample problem instance and build a simple, heuristic-based solution to understand the fundamentals before diving into more advanced optimization techniques in the upcoming notebooks.

## What is the Workforce Allocation Problem?

The Workforce Allocation Problem is a type of assignment and scheduling problem where you need to assign a set of resources (teams) to a set of jobs (tasks) while respecting a variety of constraints. It's a common challenge in workforce management, project planning, and logistics.

**Core Entities:**
* A set of $N$ **tasks** or **activities**, each with a predefined start time $s_i$ and end time $e_i$.
* A set of $M$ **teams** or **resources**, each with its own work calendar defining their windows of availability.

**Core Constraints:**
* **Unique Assignment:** Each task must be assigned to exactly one team.
* **Conflict Constraint:** If two tasks $i$ and $k$ overlap in time (i.e., their time intervals $[s_i, e_i)$ and $[s_k, e_k)$ intersect), they cannot be assigned to the same team.
* **Availability Constraint:** A team can only be assigned to a task if it is available for the *entire duration* of that task.
* **Compatibility Constraint:** Some tasks may have a restricted list of teams that are qualified to perform them (e.g., due to skill requirements).

**Objective:**
* The primary objective is typically to find a valid assignment of teams to all tasks that **minimizes the total number of unique teams used**.

### Prerequisites

These notebooks require the `discrete-optimization` library. If you are running this on Google Colab, the following cell will install it for you.

In [ ]:
# On Colab: install the library
import sys

ON_COLAB = "google.colab" in sys.modules
if ON_COLAB:
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install discrete-optimization

### Imports

In [ ]:
import logging

import networkx as nx

from discrete_optimization.datasets import fetch_data_from_cp25
from discrete_optimization.workforce.allocation.parser import (
    build_allocation_problem_from_scheduling,
)

# Import necessary classes from the discrete-optimization library
from discrete_optimization.workforce.allocation.problem import (
    TeamAllocationProblem,
    TeamAllocationSolution,
)
from discrete_optimization.workforce.allocation.utils import plot_allocation_solution
from discrete_optimization.workforce.scheduling.parser import (
    get_data_available,
    parse_json_to_problem,
)

# Set a default renderer for Plotly figures
# pio.renderers.default = "svg"


# Set logging to a basic level
logging.basicConfig(level=logging.INFO)

### Load a Problem Instance

For this problem, we'll load an instance from a JSON format that describes a more general scheduling problem. We then convert this into a `TeamAllocationProblem` instance.

In [ ]:
# Get paths to available data files
files = get_data_available()
if not files:
    fetch_data_from_cp25()
    files = get_data_available()

# We will use the second available instance for this tutorial
file_path = files[1]

# First, parse the file into a general scheduling problem
scheduling_problem = parse_json_to_problem(file_path)
# For this notebook we discard this constraint
scheduling_problem.same_allocation = []
# Next, convert it into a TeamAllocationProblem
# This helper function automatically computes the conflict and compatibility graphs
problem: TeamAllocationProblem = build_allocation_problem_from_scheduling(
    problem=scheduling_problem
)

print(f"Problem Loaded:")
print(f"Number of tasks: {problem.number_of_activity}")
print(f"Number of teams: {problem.number_of_teams}")

## Visualizing the Problem

To better understand the instance, we can visualize its key components.

### The Activity Conflict Graph
The time overlaps between tasks create a conflict graph. 
In this graph, each task is a node, and an edge is drawn between two tasks if they overlap in time. 
This means they are in conflict and must be assigned to different teams. 
This is a classic graph coloring constraint, where no two adjacent nodes can have the same color (or in our case, the same team).
You can hover over the nodes to see the task names and zoom or pan to explore the connections.

In [ ]:
import plotly.graph_objects as go

# Convert the graph to networkx for layout calculation
conflict_graph_nx = problem.graph_activity.to_networkx()
pos = nx.spring_layout(conflict_graph_nx, seed=42)

# Create the edge trace for Plotly
edge_x = []
edge_y = []
for edge in conflict_graph_nx.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo="none",
    mode="lines",
)

# Create the node trace for Plotly
node_x = []
node_y = []
node_text = []
for node in conflict_graph_nx.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(str(node))

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode="markers+text",
    hoverinfo="text",
    text=node_text,
    textposition="top center",
    marker=dict(showscale=False, color="#6495ED", size=10, line_width=2),
)

# Create the figure with a fixed square size
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title="<br>Activity Conflict Graph",
        titlefont_size=16,
        showlegend=False,
        hovermode="closest",
        margin=dict(b=20, l=5, r=5, t=40),
        width=800,  # Set width
        height=800,  # Set height to be the same as width
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    ),
)

fig.show()

### Team-Task Compatibility

Not all teams can perform all tasks, due to skill requirements or calendar availability. Let's inspect the allowed teams for a few sample tasks.

In [ ]:
tasks_to_check = ["27775", "35745"]
for task_name in tasks_to_check:
    allowed_teams = problem.compute_allowed_team_for_task(task_name)
    print(f"Task '{task_name}' can be performed by teams: {allowed_teams}\n")

In this case, the task "35745" has to be done by some specialized team.

## A Simple (Handcrafted) Solution

Before using powerful optimizers, let's create a feasible solution using a simple greedy heuristic. This will give us a baseline to compare against later.

**The Greedy Algorithm:**
1.  Sort tasks by their start time.
2.  For each task, iterate through the teams (from team 0 to team M-1).
3.  Assign the task to the *first* team that is both **allowed** (compatible and available) and **free** (not assigned to an overlapping task).


In [ ]:
def solve_greedy(problem: TeamAllocationProblem) -> TeamAllocationSolution:
    """A simple greedy solver for the Workforce Allocation Problem."""
    # Initialize allocation with -1 (unassigned)
    allocation = [-1] * problem.number_of_activity

    # Sort tasks by start time
    task_indices = sorted(
        range(problem.number_of_activity),
        key=lambda i: problem.schedule[problem.index_to_activities_name[i]][0],
    )

    for task_index in task_indices:
        task_name = problem.index_to_activities_name[task_index]
        task_start, task_end = problem.schedule[task_name]

        allowed_teams_indices = problem.compute_allowed_team_index_for_task(task_name)

        for team_index in sorted(allowed_teams_indices):
            is_team_free = True
            # Check for conflicts with tasks already assigned to this team
            for other_task_index, assigned_team in enumerate(allocation):
                if assigned_team == team_index:
                    other_task_name = problem.index_to_activities_name[other_task_index]
                    other_start, other_end = problem.schedule[other_task_name]
                    # Check for time overlap
                    if max(task_start, other_start) < min(task_end, other_end):
                        is_team_free = False
                        break

            if is_team_free:
                allocation[task_index] = team_index
                break  # Move to the next task

    # Check if all tasks were assigned
    if -1 in allocation:
        print("Warning: Greedy solver could not assign all tasks.")

    return TeamAllocationSolution(problem=problem, allocation=allocation)

In [ ]:
# Solve with our greedy heuristic
greedy_solution = solve_greedy(problem)

# Evaluate the solution
fitness = problem.evaluate(greedy_solution)
satisfy = problem.satisfy(greedy_solution)

print(f"Greedy Solution Found:")
print(f"  - Is the solution feasible? {satisfy}")
print(f"  - Number of teams used: {fitness.get('nb_teams', 'N/A')}")
print(f"  - Number of constraint violations: {fitness.get('nb_violations', 0)}")

### Visualizing the Greedy Solution

Now let's plot the resulting schedule. The plot below is a Gantt-style chart where each row represents a team, and the bars represent the tasks assigned to them.

In [ ]:
fig = plot_allocation_solution(
    problem=problem,
    sol=greedy_solution,
    title="Greedy Allocation Schedule",
    display=False,
)
fig.show()

## Conclusion

In this notebook, we've defined the Team Allocation Problem, explored its structure by visualizing its constraints, and built a simple greedy solution. This provides a solid foundation and a baseline result.

However, this greedy approach is not guaranteed to find the *optimal* solution (i.e., the one using the minimum possible number of teams). 

➡️ **Next up:** In [Notebook 2](2-Solving%20workforce%20allocation%20with%20CP.ipynb), we will formulate this problem using **Constraint Programming** to find a provably optimal solution.